In [ ]:
import numpy as np
import pandas as pd
import requests
import matplotlib.pyplot as plt

def terrain_analysis_dataframe(min_lat, max_lat, min_lon, max_lon, resolution=30):
    """
    Analyze terrain and return DataFrame with latitude, longitude, slope, and aspect
    
    Parameters:
    min_lat, max_lat: Latitude bounds
    min_lon, max_lon: Longitude bounds
    resolution: Resolution in meters
    
    Returns:
    pandas.DataFrame: DataFrame with columns for latitude, longitude, slope, and aspect
    """
    # 1. Fetch elevation data
    print(f"Fetching elevation data for coordinates: ({min_lat:.4f}, {min_lon:.4f}) to ({max_lat:.4f}, {max_lon:.4f})")
    
    # Calculate number of samples based on resolution
    lat_samples = int((max_lat - min_lat) * 111000 / resolution)
    lon_samples = int((max_lon - min_lon) * 111000 / resolution)
    
    # Ensure reasonable number of samples
    lat_samples = min(lat_samples, 100)
    lon_samples = min(lon_samples, 100)
    
    # Create latitude and longitude arrays
    lats = np.linspace(min_lat, max_lat, lat_samples)
    lons = np.linspace(min_lon, max_lon, lon_samples)
    
    # Create coordinate grid
    lat_grid, lon_grid = np.meshgrid(lats, lons)
    
    # Prepare points for API request
    points = []
    for i in range(lat_grid.shape[0]):
        for j in range(lat_grid.shape[1]):
            points.append({
                "latitude": lat_grid[i, j],
                "longitude": lon_grid[i, j]
            })
    
    # API request to get elevation data
    url = "https://api.open-elevation.com/api/v1/lookup"
    payload = {"locations": points}
    
    try:
        response = requests.post(url, json=payload)
        data = response.json()
        
        # Extract elevations and reshape to grid
        elevations = np.array([point["elevation"] for point in data["results"]])
        elevation_grid = elevations.reshape(lat_grid.shape)
        
        print(f"Successfully fetched elevation data: {lat_samples}x{lon_samples} grid")
    except Exception as e:
        print(f"Error fetching elevation data: {e}")
        print("Using synthetic data instead...")
        
        # Generate synthetic elevation data as fallback
        elevation_grid = 500 + 200 * np.random.rand(lat_grid.shape[0], lat_grid.shape[1])
        elevation_grid += 300 * np.sin(5 * lon_grid) * np.cos(5 * lat_grid)
    
    # 2. Calculate slope and aspect
    # Get approximate cell size in meters
    cell_size_y = (max_lat - min_lat) / lat_samples * 111000  # m/cell
    cell_size_x = (max_lon - min_lon) / lon_samples * 111000 * np.cos(np.radians(np.mean([min_lat, max_lat])))  # m/cell
    
    # Calculate slope (in degrees)
    dy, dx = np.gradient(elevation_grid, cell_size_y, cell_size_x)
    slope = np.degrees(np.arctan(np.sqrt(dx**2 + dy**2)))
    
    # Calculate aspect (in degrees)
    aspect = np.degrees(np.arctan2(dy, dx))
    aspect = np.mod(aspect + 360, 360)
    
    # 3. Create DataFrame
    # Flatten the arrays
    lat_flat = lat_grid.flatten()
    lon_flat = lon_grid.flatten()
    elevation_flat = elevation_grid.flatten()
    slope_flat = slope.flatten()
    aspect_flat = aspect.flatten()
    
    # Create a DataFrame with just the requested columns
    df = pd.DataFrame({
        'latitude': lat_flat,
        'longitude': lon_flat,
        'slope': slope_flat,        # in degrees (0-90)
        'aspect': aspect_flat       # in degrees (0-360, clockwise from north)
    })
    
    # Display basic statistics
    print(f"\nTerrain Statistics:")
    print(f"Average Slope: {df['slope'].mean():.1f} degrees")
    print(f"Number of points: {len(df)}")
    
    # Return the DataFrame
    return df

# Example usage
terrain_df = terrain_analysis_dataframe(
    min_lat=41.35,
    max_lat=41.45,
    min_lon=2.10,
    max_lon=2.22,
    resolution=200
)

# Display head of DataFrame
print("\nDataFrame head:")
print(terrain_df.head())

# Show dataset shape
print(f"DataFrame shape: {terrain_df.shape}")

# Save to CSV
terrain_df.to_csv('terrain_data.csv', index=False)
print("\nData saved to 'terrain_data.csv'")

Fetching elevation data for coordinates: (41.3500, 2.1000) to (41.4500, 2.2200)
Successfully fetched elevation data: 55x66 grid

Terrain Statistics:
Average Slope: 4.3 degrees
Number of points: 3630

DataFrame head:
    latitude  longitude     slope  aspect
0  41.350000        2.1  0.000000     0.0
1  41.351852        2.1  0.000000     0.0
2  41.353704        2.1  0.189237     0.0
3  41.355556        2.1  0.946101     0.0
4  41.357407        2.1  1.324425     0.0
DataFrame shape: (3630, 4)

Data saved to 'terrain_data.csv'
